In [8]:
# Cell 1: Imports
import cv2
import mediapipe as mp
import os
import numpy as np

print("All libraries imported successfully. Ready to proceed.")

All libraries imported successfully. Ready to proceed.


In [9]:
# Cell 2: Load AI Models
# Initialize MediaPipe Pose
mp_pose = mp.solutions.pose
pose = mp_pose.Pose(
    static_image_mode=False,
    model_complexity=1,
    smooth_landmarks=True,
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5
)

# Initialize MediaPipe Drawing
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles

print("MediaPipe Pose model loaded and initialized successfully.")

MediaPipe Pose model loaded and initialized successfully.


In [21]:
# Cell 3: Check and Define File Paths
# First, let's find where we are and what files exist
import glob

print("=== DIAGNOSTICS ===")
print(f"Current working directory: {os.getcwd()}")
print()

# Look for video files in common locations
print("Looking for video files...")
video_extensions = ['*.mp4', '*.avi', '*.mov', '*.MP4', '*.AVI', '*.MOV']
found_videos = []

# Search in current directory and subdirectories
for ext in video_extensions:
    found_videos.extend(glob.glob(f'**/{ext}', recursive=True))
    found_videos.extend(glob.glob(ext))

# Remove duplicates
found_videos = list(set(found_videos))

if found_videos:
    print(f"\nFound {len(found_videos)} video file(s):")
    for i, video in enumerate(found_videos, 1):
        size = os.path.getsize(video) / (1024 * 1024)  # Size in MB
        print(f"  {i}. {video} ({size:.2f} MB)")
else:
    print("\n⚠ No video files found in current directory or subdirectories")

print("\n=== SETUP ===")

# Auto-select the first found video
if found_videos:
    INPUT_VIDEO_PATH = found_videos[0]
    print(f"✓ Auto-selected video: {INPUT_VIDEO_PATH}")
else:
    INPUT_VIDEO_PATH = 'demo_shot2.mp4'
    print(f"Using default path: {INPUT_VIDEO_PATH}")

# Check if the input file exists
if os.path.exists(INPUT_VIDEO_PATH):
    print(f"✓ Input file confirmed: {INPUT_VIDEO_PATH}")
    file_size = os.path.getsize(INPUT_VIDEO_PATH) / (1024 * 1024)
    print(f"  File size: {file_size:.2f} MB")
else:
    print(f"✗ Input file NOT found: {INPUT_VIDEO_PATH}")
    print("\n>>> ERROR <<<")
    print("Video file not accessible. Please check the path.")

# Output path - save in same directory as notebook
OUTPUT_VIDEO_PATH = 'skeleton_overlay_output2.mp4'
print(f"\n✓ Output will be saved to: {OUTPUT_VIDEO_PATH}")
print("\nReady to process! Run Cell 4 next.")


=== DIAGNOSTICS ===
Current working directory: d:\Users\Anoshia\BattingEdge_FYP\notebooks

Looking for video files...

Found 3 video file(s):
  1. demo_shot2.mp4 (3.81 MB)
  2. demo_shot.mp4 (2.36 MB)
  3. skeleton_overlay_output.mp4 (1.40 MB)

=== SETUP ===
✓ Auto-selected video: demo_shot2.mp4
✓ Input file confirmed: demo_shot2.mp4
  File size: 3.81 MB

✓ Output will be saved to: skeleton_overlay_output2.mp4

Ready to process! Run Cell 4 next.


In [22]:
# Cell 1: Imports
import cv2
import mediapipe as mp
import os
import numpy as np

print("All libraries imported successfully. Ready to proceed.")

# Cell 2: Load AI Models
# Initialize MediaPipe Pose
mp_pose = mp.solutions.pose
pose = mp_pose.Pose(
    static_image_mode=False,
    model_complexity=1,
    smooth_landmarks=True,
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5
)

# Initialize MediaPipe Drawing
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles

print("MediaPipe Pose model loaded and initialized successfully.")

# Cell 3: Check and Define File Paths
# First, let's find where we are and what files exist
import glob

print("=== DIAGNOSTICS ===")
print(f"Current working directory: {os.getcwd()}")
print()

# Look for video files in common locations
print("Looking for video files...")
video_extensions = ['*.mp4', '*.avi', '*.mov', '*.MP4', '*.AVI', '*.MOV']
found_videos = []

# Search in current directory and subdirectories
for ext in video_extensions:
    found_videos.extend(glob.glob(f'**/{ext}', recursive=True))
    found_videos.extend(glob.glob(ext))

# Remove duplicates
found_videos = list(set(found_videos))

if found_videos:
    print(f"\nFound {len(found_videos)} video file(s):")
    for i, video in enumerate(found_videos, 1):
        size = os.path.getsize(video) / (1024 * 1024)  # Size in MB
        print(f"  {i}. {video} ({size:.2f} MB)")
else:
    print("\n⚠ No video files found in current directory or subdirectories")

print("\n=== SETUP ===")

# Auto-select the first found video
if found_videos:
    INPUT_VIDEO_PATH = found_videos[0]
    print(f"✓ Auto-selected video: {INPUT_VIDEO_PATH}")
else:
    INPUT_VIDEO_PATH = 'demo_shot.mp4'
    print(f"Using default path: {INPUT_VIDEO_PATH}")

# Check if the input file exists
if os.path.exists(INPUT_VIDEO_PATH):
    print(f"✓ Input file confirmed: {INPUT_VIDEO_PATH}")
    file_size = os.path.getsize(INPUT_VIDEO_PATH) / (1024 * 1024)
    print(f"  File size: {file_size:.2f} MB")
else:
    print(f"✗ Input file NOT found: {INPUT_VIDEO_PATH}")
    print("\n>>> ERROR <<<")
    print("Video file not accessible. Please check the path.")

# Output path - save in same directory as notebook
OUTPUT_VIDEO_PATH = 'skeleton_overlay_output2.mp4'
print(f"\n✓ Output will be saved to: {OUTPUT_VIDEO_PATH}")
print("\nReady to process! Run Cell 4 next.")

# Cell 4: The Core AI Pipeline
cap = None
out = None

try:
    # 1. Open the video file
    cap = cv2.VideoCapture(INPUT_VIDEO_PATH)
    
    if not cap.isOpened():
        raise Exception(f"Error: Could not open video file at {INPUT_VIDEO_PATH}")
    
    # 2. Get video properties
    frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = int(cap.get(cv2.CAP_PROP_FPS))
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    
    print(f"Video properties: {frame_width}x{frame_height} @ {fps} FPS")
    print(f"Total frames: {total_frames}")
    
    # 3. Create a VideoWriter object
    # Using 'mp4v' codec for .mp4 files
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(OUTPUT_VIDEO_PATH, fourcc, fps, (frame_width, frame_height))
    
    if not out.isOpened():
        raise Exception("Error: Could not create output video file")
    
    print("\nStarting video processing... This may take a moment.")
    
    # 4. Loop through every frame
    frame_count = 0
    
    while cap.isOpened():
        ret, frame = cap.read()
        
        if not ret:
            break
        
        frame_count += 1
        
        # Show progress every 10 frames
        if frame_count % 10 == 0 or frame_count == 1:
            progress = (frame_count / total_frames) * 100
            print(f"Processing frame {frame_count}/{total_frames} ({progress:.1f}%)...", end='\r')
        
        # 5. Convert BGR to RGB for MediaPipe
        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        
        # 6. Process the frame with MediaPipe Pose
        mp_results = pose.process(frame_rgb)
        
        # 7. Draw the skeleton if pose landmarks were detected
        if mp_results.pose_landmarks:
            # Draw landmarks and connections
            mp_drawing.draw_landmarks(
                frame,
                mp_results.pose_landmarks,
                mp_pose.POSE_CONNECTIONS,
                landmark_drawing_spec=mp_drawing_styles.get_default_pose_landmarks_style()
            )
        
        # 8. Write the frame to output video
        out.write(frame)
    
    print("\n")  # New line after progress indicator
    print(f"✓ Successfully processed {frame_count} frames")
    
except Exception as e:
    print(f"\nError occurred: {str(e)}")
    import traceback
    traceback.print_exc()
    
finally:
    # 9. Release resources
    if cap is not None:
        cap.release()
    if out is not None:
        out.release()
    cv2.destroyAllWindows()
    
    print(f"\n--- PROCESSING COMPLETE ---")
    
    # Check if output file was created successfully
    if os.path.exists(OUTPUT_VIDEO_PATH):
        file_size = os.path.getsize(OUTPUT_VIDEO_PATH) / (1024 * 1024)  # Size in MB
        print(f"✓ Video saved successfully at: {OUTPUT_VIDEO_PATH}")
        print(f"✓ File size: {file_size:.2f} MB")
        print(f"✓ Total frames processed: {frame_count}")
    else:
        print(f"✗ Output file was not created")

All libraries imported successfully. Ready to proceed.
MediaPipe Pose model loaded and initialized successfully.
=== DIAGNOSTICS ===
Current working directory: d:\Users\Anoshia\BattingEdge_FYP\notebooks

Looking for video files...

Found 3 video file(s):
  1. demo_shot2.mp4 (3.81 MB)
  2. demo_shot.mp4 (2.36 MB)
  3. skeleton_overlay_output.mp4 (1.40 MB)

=== SETUP ===
✓ Auto-selected video: demo_shot2.mp4
✓ Input file confirmed: demo_shot2.mp4
  File size: 3.81 MB

✓ Output will be saved to: skeleton_overlay_output2.mp4

Ready to process! Run Cell 4 next.
Video properties: 1920x1080 @ 60 FPS
Total frames: 89

Starting video processing... This may take a moment.
Processing frame 80/89 (89.9%)...

✓ Successfully processed 89 frames

--- PROCESSING COMPLETE ---
✓ Video saved successfully at: skeleton_overlay_output2.mp4
✓ File size: 3.87 MB
✓ Total frames processed: 89
